In [10]:
import pandas as pd
import joblib
import argparse

In [11]:
loaded_df, spamProbality = joblib.load('../Models/BayesienModel.joblib')

In [12]:
def TestPhraseList(phraseList,spamProba=1):
    for word in phraseList:
        vraisemblance = loaded_df["vraisemblance"].get(word,"NotFound")
        if vraisemblance!="NotFound":
            spamProba*=vraisemblance
    return(spamProba)

def TestPhrase(phrase:str,spamProba=1):
    phraseList=phrase.split()
    return TestPhraseList(phraseList,spamProba=spamProba)


In [13]:
testDF = pd.read_csv(
    "SMSSpamCollection.txt", delimiter="\t", header=None, names=["spam", "text"]
)

In [14]:
testDF["text"]=testDF["text"].str.split()
testDF["spam"]=testDF["spam"].apply(str.lower)
testDF["result"]=testDF["text"].apply(TestPhraseList,spamProba=spamProbality)
testDF["Finalresult"]=['spam' if x > 1 else 'ham' for x in testDF['result']]

In [15]:
wrong = len(testDF[testDF["Finalresult"]!=testDF["spam"]].index)
FP = len(testDF[(testDF["Finalresult"]=="spam")&(testDF["spam"]=="ham")].index)
FN = len(testDF[(testDF["Finalresult"]=="ham")&(testDF["spam"]=="spam")].index)
VP = len(testDF[(testDF["Finalresult"]=="ham")&(testDF["spam"]=="ham")].index)
VN = len(testDF[(testDF["Finalresult"]=="spam")&(testDF["spam"]=="spam")].index)
wrong/len(testDF.index)
exactitude = (VN+VP)/(VN+FN+FP+VP)
print(f"Exactitude : {exactitude}")
rappel = VP /(VP+FN)
print(f"Rappel : {rappel}")
print(f"Rappel négative: {VN/(VN+FP)}")
precision = VP/(VP+FP)
print(f"Precision : {precision}")
print(f"Précision négative: {VN/(VN+FN)}")
F1Score = (2 * precision * rappel) / (precision + rappel)
print(f"F1-Score : {F1Score}")
print(f"F1-Score Négatif: {(2 * VN/(VN+FN) * VN/VN+FP )/ (VN/(VN+FN) + VN/VN+FP)}")

Exactitude : 0.9800789662598708
Rappel : 0.9972573839662447
Rappel négative: 0.8822115384615384
Precision : 0.9796891191709844
Précision négative: 0.9825970548862115
F1-Score : 0.988395190799791
F1-Score Négatif: 0.9998259402573406
